<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_007_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Evaluación y ajuste de hiperparámetros</font>**

<p align="justify">
♥ En el Colab anterior, vimos dos enfoques para ajustar los hiperparámetros. Sin embargo, no se presento un marco adecuado para evaluar los modelos ajustados. En su lugar, nos concentramos en el mecanismo utilizado para encontrar el mejor conjunto de parámetros.
<br><br>
En este Colab, reutilizaremos algunos conocimientos para mostrar cómo evaluar modelos en los que es necesario ajustar los hiperparámetros. Por eso, primero cargaremos el dataset y crearemos el modelo predictivo que queremos optimizar y posteriormente evaluar.

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

 ## **<font color="DeepPink">Carga del conjunto de datos</font>**

In [ ]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")
adult_census.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


 ## **<font color="DeepPink">Separamos la variable objetivo y las variables explicativas</font>**

<p align="justify">
👀 Asignamos a un objeto la variable objetivo:
</p>


In [ ]:
target_name = "class"
y = adult_census[target_name]
y

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

<p align="justify">
👀 Eliminamos de nuestros datos la variable objetivo y la columna <code>education-num</code> que duplica la información de la columna <code>education</code>.

In [ ]:
X = adult_census.drop(columns=[target_name, "education-num"])
X.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


 ## **<font color="DeepPink">Conjunto de entrenamiento y conjunto de prueba</font>**

👀 Dividimos en conjunto de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

 # **<font color="DeepPink">Nuestro modelo predictivo</font>**

<p align="justify">
👀 Ahora creamos el modelo predictivo que queremos optimizar. Tenga en cuenta que este pipeline es idéntico al que usamos en el Colab anterior.

<p align="justify">
✅ Definiremos un Pipeline que va a manejar características categóricas.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

In [ ]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

In [ ]:
categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1)

In [ ]:
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor, categorical_columns),],
    remainder='passthrough',
    sparse_threshold=0,)

In [ ]:
preprocessor

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('cat_preprocessor',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

<p align="justify">
✅ Usamos un clasificador basado en árboles (histogram gradient-boosting) para predecir si una persona gana o no más de 50 k$ al año.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [ ]:
model = Pipeline([("preprocessor", preprocessor),
                  ("classifier", HistGradientBoostingClassifier(random_state=42,
                                                                max_leaf_nodes=4)),])

In [ ]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

 # **<font color="DeepPink">Evaluación del modelo</font>**

 ## **<font color="DeepPink">Sin ajuste de hiperparámetros</font>**

<p align="justify">
Anteriormente vimos que se debe usar la validación cruzada para evaluar el modelo. La validación cruzada permite obtener una distribución de las puntuaciones del modelo. Así, teniendo esta distribución a mano, podemos llegar a evaluar la variabilidad de nuestra estimación del rendimiento de generalización del modelo.
<br><br>
Ahora recordamos las herramientas de <code>scikit-learn</code> necesarias para obtener la media y la desviación estándar de las puntuaciones.

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cv_results = cross_validate(model, X, y, cv=5)
cv_results = pd.DataFrame(cv_results)
cv_results

,fit_time,score_time,test_score
0,0.581936,0.087895,0.863241
1,0.529134,0.105229,0.860784
2,0.531778,0.084779,0.860360
3,0.558694,0.087248,0.862408
4,0.669932,0.161379,0.866912


In [ ]:
print("")
print("Generalization score without hyperparameters tuning:\n"
     f"{cv_results['test_score'].mean():.3f} ± {cv_results['test_score'].std():.3f}")


Generalization score without hyperparameters tuning:
0.863 ± 0.003


 ## **<font color="DeepPink">Con ajuste de hiperparámetros</font>**

<p align="justify">
👀 Ahora utilizaremos una estrategia de grid-search y reproduciremos los pasos realizados anteriormente. Primero, tenemos que incrustar nuestro modelo en grid-search y especificar los parámetros y los valores de los parámetros que queremos explorar.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'classifier__learning_rate': (0.05, 0.5),
              'classifier__max_leaf_nodes': (10, 30),}

In [ ]:
model_grid_search = GridSearchCV(model,
                                 param_grid=param_grid,
                                 n_jobs=2,
                                 cv=2)

In [ ]:
model_grid_search.fit(X, y)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          sparse_threshold=0,
                                                          transformers=[('cat_preprocessor',
                                                                         OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                        unknown_value=-1),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('classifier',
                                        HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                                       random_state=42))]),
             n_jobs=2,
             param_grid={'classifier__learning_rate': (0.05, 0.5),
                         'classifier__max_leaf_nodes': (10, 30)})

<p align="justify">
Como se vio anteriormente, al llamar al método de ajuste, el modelo incrustado en grid-search se entrena con todas las combinaciones posibles de parámetros resultantes de la cuadrícula de parámetros. La mejor combinación se selecciona manteniendo la combinación que conduce a la mejor puntuación media de validación cruzada.

In [ ]:
cv_results = pd.DataFrame(model_grid_search.cv_results_)
cv_results[[
    "param_classifier__learning_rate",
    "param_classifier__max_leaf_nodes",
    "mean_test_score",
    "std_test_score",
    "rank_test_score"]]

,param_classifier__learning_rate,param_classifier__max_leaf_nodes,mean_test_score,std_test_score,rank_test_score
0,0.05,10,0.864195,0.000061,4
1,0.05,30,0.870910,0.000061,1
2,0.5,10,0.869743,0.000532,2
3,0.5,30,0.866058,0.001515,3


In [ ]:
model_grid_search.best_params_

{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}

<p align="justify">
🛑 Una advertencia importante aquí se refiere a la evaluación del rendimiento de la generalización. De hecho, la media y la desviación estándar de las puntuaciones calculadas por la validación cruzada en grid-search no son potencialmente buenas estimaciones del rendimiento de generalización que obtendríamos al reajustar un modelo con la mejor combinación de valores de hiperparámetros en el conjunto de datos completo.
<br><br>
Tenga en cuenta que <code>scikit-learn</code> realiza automáticamente este ajuste de forma predeterminada cuando llama a <code>model_grid_search.fit.</code> Este modelo reacondicionado se entrena con más datos que los diferentes modelos entrenados internamente durante la validación cruzada de la búsqueda en cuadrícula.
<br><br>
Por lo tanto, utilizamos el conocimiento del conjunto de datos completo para decidir los hiperparámetros de nuestro modelo y para entrenar el modelo reajustado. Debido a lo anterior, se debe mantener un equipo de prueba externo para la evaluación final del modelo reacondicionado. Destacamos aquí el proceso que utiliza una sola división de entrenamiento y prueba.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
model_grid_search.fit(data_train, target_train)
accuracy = model_grid_search.score(data_test, target_test)
print("")
print(f"Accuracy on test set: {accuracy:.3f}")


Accuracy on test set: 0.877


<p align="justify">
La medida de puntuación en el conjunto de prueba final está casi dentro del rango de la puntuación CV interna para la mejor combinación de hiperparámetros. Esto es tranquilizador, ya que significa que el procedimiento de ajuste no causó un sobreajuste significativo en sí mismo (de lo contrario, la puntuación final de la prueba habría sido más baja que las puntuaciones CV internas).
<br><br>
Eso era de esperar porque nuestra búsqueda en cuadrícula exploró muy pocas combinaciones de hiperparámetros en aras de la velocidad. La puntuación de la prueba del modelo final es en realidad un poco más alta de lo que podríamos haber esperado de la validación cruzada interna.
<br><br>
Esto también se espera porque el modelo reajustado se entrena en un conjunto de datos más grande que los modelos evaluados en el ciclo de CV interno del procedimiento de grid-search. Este suele ser el caso de que los modelos entrenados en un mayor número de muestras tienden a generalizar mejor.
<br><br>
Anteriormente la selección de los mejores hiperparámetros se realizó solo en el conjunto de entrenamiento de la división de prueba y entrenamiento inicial. Luego, evaluamos el rendimiento de generalización de nuestro modelo ajustado en el conjunto de prueba omitido. Esto se puede mostrar esquemáticamente de la siguiente manera:

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Validacion-002.png?raw=true" width="600">
</p>


<p align="justify">
<b>Nota</b>: Esta figura muestra el caso particular de la estrategia de validación cruzada <code>K-fold</code> usando <code>n_splits = 5</code> para dividir aún más el conjunto de entrenamiento que proviene de una división de prueba de entrenamiento. Para cada división de validación cruzada, el procedimiento entrena un modelo en todas las muestras rojas, evalúa la puntuación de un conjunto dado de hiperparámetros en las muestras verdes. Los mejores hiperparámetros se seleccionan en función de esas puntuaciones intermedias.
<br><br>
Luego, se ajusta un modelo final sintonizado con esos hiperparámetros en la concatenación de las muestras roja y verde y se evalúa en las muestras azules.
Las muestras verdes a veces se denominan conjuntos de validación para diferenciarlos del conjunto de prueba final en azul.
<br><br>
Sin embargo, esta evaluación solo nos proporciona una estimación puntual única del rendimiento de la generalización. Como recordamos al comienzo de este Colab, es beneficioso tener una idea aproximada de la incertidumbre de nuestro rendimiento de generalización estimado. Por lo tanto, deberíamos usar una validación cruzada adicional para esta evaluación.
<br><br>
Este patrón se denomina validación cruzada anidada. Usamos una validación cruzada interna para la selección de los hiperparámetros y una validación cruzada externa para la evaluación del rendimiento de generalización del modelo ajustado reajustado.
<br><br>
En la práctica, solo necesitamos incrustar en grid-search la función <code>cross_validate</code> para realizar dicha evaluación.

In [ ]:
cv_results = cross_validate(
    model_grid_search, X, y, cv=5, n_jobs=2, return_estimator=True)

In [ ]:
cv_results = pd.DataFrame(cv_results)
cv_test_scores = cv_results['test_score']
print("")
print(
    "Generalization score with hyperparameters tuning:\n"
    f"{cv_test_scores.mean():.3f} ± {cv_test_scores.std():.3f}")


Generalization score with hyperparameters tuning:
0.871 ± 0.003


<p align="justify">
Este resultado es compatible con la puntuación de la prueba medida con la división de prueba y prueba exterior. Sin embargo, en este caso, podemos aprender la variabilidad de nuestra estimación del rendimiento de generalización gracias a la medida de la desviación estándar de las puntuaciones medidas en la validación cruzada externa.
<br><br>
Aquí hay una representación esquemática del procedimiento completo de validación cruzada anidada:

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Validacion-003.png?raw=true" width="600">
</p>


<p align="justify">
<b>Nota</b>: Esta figura ilustra la estrategia de validación cruzada anidada usando <code>cv_inner = KFold(n_splits=4)</code> y <code>cv_outer = KFold(n_splits=5)</code>.
<br><br>
Para cada división interna de validación cruzada (indexada en el lado izquierdo), el procedimiento entrena un modelo en todas las muestras rojas y evalúa la calidad de los hiperparámetros en las muestras verdes.
<br><br>
Para cada división externa de validación cruzada (indexada en el lado derecho), se seleccionan los mejores hiperparámetros en función de los puntajes de validación (calculados en las muestras de codicia) y se reajusta un modelo en la concatenación de las muestras roja y verde.
<br><br>
A continuación, se evalúa el rendimiento de generalización de los 5 modelos reacondicionados del bucle CV externo en las muestras azules para obtener las puntuaciones finales. Además, pasando el parámetro <code>return_estimator=True</code>, podemos comprobar el valor de los mejores hiperparámetros obtenidos para cada pliegue de la validación cruzada exterior.

In [ ]:
for cv_fold, estimator_in_fold in enumerate(cv_results["estimator"]):
    print(f"Best hyperparameters for fold #{cv_fold + 1}:\n"
          f"{estimator_in_fold.best_params_}")

Best hyperparameters for fold #1:
{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}
Best hyperparameters for fold #2:
{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}
Best hyperparameters for fold #3:
{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}
Best hyperparameters for fold #4:
{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}
Best hyperparameters for fold #5:
{'classifier__learning_rate': 0.05, 'classifier__max_leaf_nodes': 30}


<p align="justify">
Es interesante ver si el procedimiento de ajuste de hiperparámetros siempre selecciona valores similares para los hiperparámetros. Si es el caso, entonces todo está bien. Significa que podemos implementar un ajuste de modelo con esos hiperparámetros y esperar que tenga un rendimiento predictivo real cercano al que medimos en la validación cruzada externa.
<br><br>
Pero también es posible que algunos hiperparámetros no importen en absoluto y, como resultado, en diferentes sesiones de ajuste den resultados diferentes. En este caso, cualquier valor servirá. Normalmente, esto se puede confirmar haciendo un gráfico de coordenadas paralelas de los resultados de una gran búsqueda de hiperparámetros como se ve en los ejercicios.
<br><br>
Desde el punto de vista de la implementación, también se podría optar por implementar todos los modelos encontrados por el ciclo externo de validación cruzada y hacer que voten para obtener las predicciones finales. Sin embargo, esto puede causar problemas operativos porque usa más memoria y hace que la predicción sea más lenta, lo que resulta en un mayor uso de recursos computacionales por predicción.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.<br>
✅ Generamos un Pipeline.
<br>
✅ Optimizamos los hiperparámetros de un modelo predictivo.
<br>
✅ Vimos cómo evaluar el rendimiento predictivo de un modelo con hiperparámetros ajustados mediante el procedimiento de validación cruzada anidada..
<br><br>


<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
